<h1>Extracting and Visualizing Stock Data</h1>
<h2>Description</h2>


Extracting essential data from a dataset and displaying it is a necessary part of data science; therefore individuals can make correct decisions based on the data. Here some stock data is extracted, and will then be displayed in a graph.

Installing packages:

In [1]:
!pip install yfinance
#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

     |████████████████████████████████| 6.4 MB 7.7 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


Importing libraries:

In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define Graphing Function


In this section, we define the function `make_graph`. It takes a dataframe with stock data, a dataframe with revenue data, and the name of the stock.

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Using yfinance to Extract Stock Data

Using the Ticker function, entering the ticker symbol of the stock Tesla, its ticker symbol is TSLA.

In [4]:
tesla = yf.Ticker("TSLA")

Using the ticker object and the function history to extract stock information and save it in a dataframe named tesla_data and displaying the first five rows of it using the head function.

In [5]:
tesla_data= tesla.history(period= "max")
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,3.800,5.000,3.508,4.778,93831500,0,0.0
1,2010-06-30,5.158,6.084,4.660,4.766,85935500,0,0.0
2,2010-07-01,5.000,5.184,4.054,4.392,41094000,0,0.0
3,2010-07-02,4.600,4.620,3.742,3.840,25699000,0,0.0
4,2010-07-06,4.000,4.000,3.166,3.222,34334500,0,0.0


## Using Webscraping to Extract Tesla Revenue Data

Using the requests library to download the webpage https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue. Parsing the html data using beautiful_soup. Using BeautifulSoup function to extract the table with Tesla Quarterly Revenue and store it into a dataframe named tesla_revenue. Finally printing the last 5 row of the tesla_revenue dataframe.

In [6]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

html_data  = requests.get(url).text

soup = BeautifulSoup(html_data, 'html5lib')

all_tables = soup.find_all('table', attrs={'class': 'historical_data_table table'})

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for table in all_tables:
    if table.find('th').getText().startswith("Tesla Quarterly Revenue"):
        for row in table.find_all("tr"):
            col = row.find_all("td")  
            if len(col) == 2: 
                date = col[0].text
                revenue = col[1].text.replace('$', '').replace(',', '')
                tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)

tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

tesla_revenue.tail()


,Date,Revenue
44,2010-09-30,31
45,2010-06-30,28
46,2010-03-31,21
48,2009-09-30,46
49,2009-06-30,27


## Using yfinance to Extract Stock Data

Using the Ticker function, entering the ticker symbol of GameStop, its ticker symbol is GME

In [7]:
gme = yf.Ticker("GME")

Using the ticker object and the function history to extract stock information and save it in a dataframe named gme_data and displaying the first five rows of it using the head function.

In [8]:
gme_data = gme.history(period= "max")
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13,6.480512,6.773398,6.413182,6.766665,19054000,0.0,0.0
1,2002-02-14,6.850830,6.864296,6.682505,6.733002,2755400,0.0,0.0
2,2002-02-15,6.733002,6.749834,6.632007,6.699337,2097400,0.0,0.0
3,2002-02-19,6.665671,6.665671,6.312188,6.430016,1852600,0.0,0.0
4,2002-02-20,6.463681,6.648838,6.413183,6.648838,1723200,0.0,0.0


## Using Webscraping to Extract GME Revenue Data

Use the requests library to download the webpage https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue.Parsing the html data using beautiful_soup. Using BeautifulSoup function to extract the table with GME Quarterly Revenue and store it into a dataframe named gme_revenue. Finally printing the last 5 row of the gme_revenue dataframe.

In [9]:
url  = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

html_data  = requests.get(url).text

soup= BeautifulSoup (html_data, 'html5lib')
all_tables = soup.find_all('table', attrs={'class': 'historical_data_table table'})

gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for table in all_tables:
    if table.find('th').getText().startswith("GameStop Quarterly Revenue"):
        for row in table.find_all("tr"):
            col = row.find_all("td")  
            if len(col) == 2: 
                date = col[0].text
                revenue = col[1].text.replace('$', '').replace(',', '')
                gme_revenue = gme_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)

gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")

gme_revenue.dropna(inplace=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

gme_revenue.tail()

,Date,Revenue
63,2006-01-31,1667
64,2005-10-31,534
65,2005-07-31,416
66,2005-04-30,475
67,2005-01-31,709


## Plotting Tesla Stock Graph:

Using the `make_graph` function to graph the Tesla Stock Data. 

In [10]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

## Plotting GameStop Stock Graph

Use the `make_graph` function to graph the GameStop Stock Data.

In [11]:
make_graph(gme_data, gme_revenue, 'GameStop')

## Summary

The stock data of GameStop and Tesla was web scraped and then plotted on a graph by using several functions and libraries.